In [1]:
import os
import random
import json
import pandas as pd
import glob
import math
import numpy
from collections import defaultdict

In [2]:
from ipynb.fs.full. Model import *

In [4]:
def implementModel(delta, path):
    
    vehicle_list, object_list = getData(path)
    vehicle_df, object_df, nObjects = getDf(vehicle_list, object_list)
    coverage = calcObjectCoverage(vehicle_df, object_df, nObjects)
    #remove
    initialLoad = 0#loadCalculator(set(vehicle_df['Id']),coverage, object_df)
    leftVehicle = NoOfObjects(vehicle_df, object_df, coverage)
    optimisedSet, objectSet = greedyAppx(leftVehicle, delta)
    finalLoad = 0#loadCalculator(optimisedSet, coverage, object_df)
    
    return vehicle_df, object_df, coverage, initialLoad, finalLoad, optimisedSet, objectSet

In [2]:
def getStats(optimisedSet, objectSet, vehicle_df, object_df):
    
    uncovered_objects, optimised_coordinates, covered_objects = [], [], []
    for index in vehicle_df.index:
        if vehicle_df.iloc[index]["Id"] in list(optimisedSet):
            optimised_coordinates.append([vehicle_df.iloc[index]["x_coordinate"], vehicle_df.iloc[index]["y_coordinate"], vehicle_df.iloc[index]["radius"]])

    for index in object_df.index:
        if object_df.iloc[index]["Id"] in list(objectSet):
            covered_objects.append([object_df.iloc[index]["x_coordinate"], object_df.iloc[index]["y_coordinate"]])
        else:
            uncovered_objects.append([object_df.iloc[index]["x_coordinate"], object_df.iloc[index]["y_coordinate"]])
    vehicle_coordinates = vehicle_df[["x_coordinate", "y_coordinate"]].values.tolist()
    
    return uncovered_objects, covered_objects, optimised_coordinates, vehicle_coordinates

In [3]:
def plot(uncovered_objects, covered_objects, optimised_coordinates, vehicle_coordinates):
    from matplotlib import pyplot as plt, patches

    fig = plt.figure(figsize=(15,10))

    for p, q in vehicle_coordinates:
        plt.scatter(p, q, color="green")

    for x,y,r in optimised_coordinates:
        ax = fig.add_subplot()
        plt.scatter(x, y, color="red")
        ax.add_patch(patches.Circle((x, y), radius=r, color='blue', fill=False))

    for p, q in uncovered_objects:
        plt.scatter(p, q, color="black", marker = "v")

    for p, q in covered_objects:
        plt.scatter(p, q, color="orange", marker = "x")

    x = [1, 100, 100]
    y = [200, 200, 350]
    plt.plot(x, y, color = "grey")
    x = [150, 150, 250]
    y = [350, 200, 200]
    plt.plot(x, y, color = "grey")
    x = [1, 100, 100]
    y = [150, 150, 0]
    plt.plot(x, y, color = "grey")
    x = [150, 150, 250]
    y = [0, 150, 150]
    plt.plot(x, y, color = "grey")

    ax.axis('equal')
    plt.show()